In [1]:
import pandas as pd
import json
import requests
% matplotlib inline

#### 上交所网站提取个股即时及历史行情

个股行情

In [12]:
pd.read_json(r'http://yunhq.sse.com.cn:32041/v1/sh1/snap/600085',encoding='gb2312').head()

,code,date,snap,time
0,600085,20170728,同仁堂,151508
1,600085,20170728,32.63,151508
2,600085,20170728,32.6,151508
3,600085,20170728,33.03,151508
4,600085,20170728,32.43,151508


个股历史行情

In [32]:
url2='http://yunhq.sse.com.cn:32041/v1/sh1/dayk/600085?begin=-5000&end=-1'

In [82]:
k=pd.read_json(url2).kline

In [83]:
df=pd.DataFrame(k.to_dict(),).T

In [85]:
df.columns=['date','open','high','low','close','vol','amt']

In [107]:
df.head()

,date,open,high,low,close,vol,amt
1970-01-01 00:00:00.019970625,19970625.0,17.00,18.18,16.50,17.22,34095500.0,587851000.0
1970-01-01 00:00:00.019970626,19970626.0,16.98,18.94,16.52,18.21,11213100.0,201651000.0
1970-01-01 00:00:00.019970627,19970627.0,18.18,18.90,17.51,17.78,8830500.0,160882000.0
1970-01-01 00:00:00.019970702,19970702.0,17.85,18.37,16.00,16.60,4605700.0,77620000.0
1970-01-01 00:00:00.019970703,19970703.0,16.60,16.66,15.20,15.38,3510700.0,54962000.0


上交所所有股票即时行情

In [42]:
url3='http://yunhq.sse.com.cn:32041/v1/sh1/list/exchange/equity?select=\
code%2Cname%2Copen%2Chigh%2Clow%2Clast%2Cprev_close%2Cchg_rate%2Cvolume%2Camount%2Ctradephase%2Cchange%2Camp_rate&\
order=&begin=1&end=1000'

In [ ]:
json_str=requests.get(url3).text
data=json.loads(json_str)['list']
pd.read_json(data).head()

#### 上海期货交易所每日行情数据

In [3]:
import requests
import json
import sqlite3
con=sqlite3.connect(r'd:\aliyun\www\my_flask\marketPrice\FutureMarketPrice.db')

In [4]:
def shfe_price(date='20170103'):
    url='http://www.shfe.com.cn/data/dailydata/kx/kx%s.dat'%date
    json_str=requests.get(url).text
    data=json.loads(json_str)['o_curinstrument']
    df=pd.DataFrame(data)
    df['date']=date
    return df

In [17]:
def price_to_sql(start='20170101'):
    df=shfe_price(start)
    for i in pd.bdate_range(start=start,periods=180):
        date=i.strftime('%Y%m%d')
        print(date)
        try:
            data=shfe_price(date=date)
            df=pd.concat([data,df])
        except:
            print('Error occured!%s'%date)
    CODE=(df.PRODUCTID.str.upper().str.strip().str[:-2]+df.DELIVERYMONTH)
    df['code']=CODE
    df=df.loc[:,['code','date', 'OPENPRICE','HIGHESTPRICE','LOWESTPRICE','CLOSEPRICE','SETTLEMENTPRICE','VOLUME','OPENINTEREST']].copy()
    df.columns=['code','date', 'open','high','low','close','settle','vol','oi']
    df=df[df.settle!=""]
    df.to_sql('FutureMarketPrice',con,if_exists='append',index=False)
    return df
    

In [18]:
a=price_to_sql('20170213')

20170213
20170214
20170215
20170216
20170217
20170220
20170221
20170222
20170223
20170224
20170227
20170228
20170301
20170302
20170303
20170306
20170307
20170308
20170309
20170310
20170313
20170314
20170315
20170316
20170317
20170320
20170321
20170322
20170323
20170324
20170327
20170328
20170329
20170330
20170331
20170403
Error occured!20170403
20170404
Error occured!20170404
20170405
20170406
20170407
20170410
20170411
20170412
20170413
20170414
20170417
20170418
20170419
20170420
20170421
20170424
20170425
20170426
20170427
20170428
20170501
Error occured!20170501
20170502
20170503
20170504
20170505
20170508
20170509
20170510
20170511
20170512
20170515
20170516
20170517
20170518
20170519
20170522
20170523
20170524
20170525
20170526
20170529
Error occured!20170529
20170530
Error occured!20170530
20170531
20170601
20170602
20170605
20170606
20170607
20170608
20170609
20170612
20170613
20170614
20170615
20170616
20170619
20170620
20170621
20170622
20170623
20170626
20170627
20170628
201

In [268]:
b=pd.read_html(r'http://www.shfe.com.cn/statements/dataview.html?paramid=delaymarket_all')

In [305]:
def shfe_stock(date='20170602'):
    url='http://www.shfe.com.cn/data/dailydata/%sweeklystock.dat'%date
    json_str=requests.get(url).text
    data=json.loads(json_str)['o_cursor']
    df=pd.DataFrame(data)
    df['date']=date
    return df

In [306]:
shfe_stock().head(1)

,PRESPOTWGHTS,PREWHSTOCKS,PREWRTWGHTS,REGNAME,REGSORT,ROWORDER,ROWSTATUS,SPOTCHANGE,SPOTWGHTS,VARNAME,VARSORT,WGHTUNIT,WHABBRNAME,WHROWS,WHSTOCKCHANGE,WHSTOCKS,WRTCHANGE,WRTWGHTS,date
0,8825,54278,5722,上海$$Shanghai,0,15,0,-689,8136,铜$$COPPER,0,2,期晟公司$$Qisheng,12,0,54278,0,5722,20170602


In [309]:
def stock_to_sql(start='20170602'):
    df=shfe_stock(start)
    import time
    for i in pd.bdate_range(start=start,freq='-7D',periods=20):
        date=i.strftime('%Y%m%d')
        print(date)
        time.sleep(1)
        try:
            data=shfe_stock(date=date)
            df=pd.concat([data,df])
        except:
            print('Error occured!%s'%date)
#     CODE=(df.PRODUCTID.str.upper().str.strip().str[:-2]+df.DELIVERYMONTH)
#     df['code']=CODE
#     df=df.loc[:,['code','date', 'OPENPRICE','HIGHESTPRICE','LOWESTPRICE','CLOSEPRICE','SETTLEMENTPRICE','VOLUME','OPENINTEREST']].copy()
#     df.columns=['code','date', 'open','high','low','close','settle','vol','oi']
#     df=df[df.settle!=""]
#     df.to_sql('FutureMarketPrice',con,if_exists='append',index=False)
    return df

In [12]:
#stock_to_sql('20170602')

In [13]:
pd.bdate_range(start='20170101',periods=138)

DatetimeIndex(['2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05',
               '2017-01-06', '2017-01-09', '2017-01-10', '2017-01-11',
               '2017-01-12', '2017-01-13',
               ...
               '2017-06-29', '2017-06-30', '2017-07-03', '2017-07-04',
               '2017-07-05', '2017-07-06', '2017-07-07', '2017-07-10',
               '2017-07-11', '2017-07-12'],
              dtype='datetime64[ns]', length=138, freq='B')

In [19]:
u=pd.read_clipboard()

In [24]:
u.columns=['code','code2','unit']

In [32]:
u1=u.drop_duplicates('code').dropna()

In [34]:
u1['type']=u.code.str[:-4]

In [41]:
u1.drop_duplicates('type')

,code,code2,unit,type
0,AG1401,AG1401,15.003471,AG
2939,AL1401,AL1401,5.000590,AL
5879,AU1401,AU1401,1000.125418,AU
7740,BU1402,BU1402,10.000050,BU
10670,CU1401,CU1401,5.000351,CU
16326,HC1407,HC1407,10.005562,HC
18542,PB1401,PB1401,5.000191,PB
21482,RB1401,RB1401,10.000641,RB
24422,RU1401,RU1401,10.002212,RU
26958,WR1404,WR1404,10.000569,WR


In [6]:
import pyecharts as pe

In [35]:
c=list(d.hot);v=list(d.amt);

In [33]:
d=pd.read_clipboard()

In [37]:
from pyecharts import EffectScatter

v1 = [10, 20, 30, 40, 50, 60]
v2 = [25, 20, 15, 10, 60, 33]
es = EffectScatter("动态散点图示例")
es.add("effectScatter", c, v)
es.render_notebook()

In [45]:
a=pd.read_clipboard()
a.head()

,code,hot,amt
0,I1709,1.244605,1.125392e+07
1,JD1709,2.645573,2.546914e+06
2,M1709,0.449245,2.524910e+06
3,L1709,1.206029,2.124855e+06
4,PP1709,0.585891,1.751668e+06


In [46]:
code=list(a.code);hot=list(a.hot);amt=list(a.amt)

In [49]:
from pyecharts import Bar
a=50;b=20;c=36;d=10;e=75;f=90

bar = Bar("期货品种投机度", "按成交量逆序排列")
bar.add("投机度",code,hot,mark_line=['average'])
bar.show_config()
bar.render_notebook()

{'_index_flag': 1017224,
 'backgroundColor': '#fff',
 'color': ['#c23531',
           '#2f4554',
           '#61a0a8',
           '#d48265',
           '#749f83',
           '#ca8622',
           '#bda29a',
           '#6e7074',
           '#546570',
           '#c4ccd3',
           '#f05b72',
           '#ef5b9c',
           '#f47920',
           '#905a3d',
           '#fab27b',
           '#2a5caa',
           '#444693',
           '#726930',
           '#b2d235',
           '#6d8346',
           '#ac6767',
           '#1d953f',
           '#6950a1',
           '#918597',
           '#f6f5ec'],
 'legend': [{'data': ['投机度'],
             'left': 'center',
             'orient': 'horizontal',
             'selectedMode': True,
             'show': True,
             'top': 'top'}],
 'series': [{'data': [1.2446047378951102,
                      2.6455728544230199,
                      0.44924452760341205,
                      1.2060291505909699,
                      0.58589147578626

In [51]:
import random

attr = ["{}天".format(i) for i in range(30)]
v1 = [random.randint(1, 30) for _ in range(30)]
bar = Bar("Bar - datazoom - slider 示例")
bar.add("", attr, v1, is_label_show=True, is_datazoom_show=True)
#bar.show_config()
bar.render_notebooktebook()

In [54]:
from pyecharts import Geo

data = [
    ("海门", 9),("鄂尔多斯", 12),("招远", 12),("舟山", 12),("齐齐哈尔", 14),("盐城", 15),
    ("赤峰", 16),("青岛", 18),("乳山", 18),("金昌", 19),("泉州", 21),("莱西", 21),
    ("日照", 21),("胶南", 22),("南通", 23),("拉萨", 24),("云浮", 24),("梅州", 25),
    ("文登", 25),("上海", 25),("攀枝花", 25),("威海", 25),("承德", 25),("厦门", 26),
    ("汕尾", 26),("潮州", 26),("丹东", 27),("太仓", 27),("曲靖", 27),("烟台", 28),
    ("福州", 29),("瓦房店", 30),("即墨", 30),("抚顺", 31),("玉溪", 31),("张家口", 31),
    ("阳泉", 31),("莱州", 32),("湖州", 32),("汕头", 32),("昆山", 33),("宁波", 33),
    ("湛江", 33),("揭阳", 34),("荣成", 34),("连云港", 35),("葫芦岛", 35),("常熟", 36),
    ("东莞", 36),("河源", 36),("淮安", 36),("泰州", 36),("南宁", 37),("营口", 37),
    ("惠州", 37),("江阴", 37),("蓬莱", 37),("韶关", 38),("嘉峪关", 38),("广州", 38),
    ("延安", 38),("太原", 39),("清远", 39),("中山", 39),("昆明", 39),("寿光", 40),
    ("盘锦", 40),("长治", 41),("深圳", 41),("珠海", 42),("宿迁", 43),("咸阳", 43),
    ("铜川", 44),("平度", 44),("佛山", 44),("海口", 44),("江门", 45),("章丘", 45),
    ("肇庆", 46),("大连", 47),("临汾", 47),("吴江", 47),("石嘴山", 49),("沈阳", 50),
    ("苏州", 50),("茂名", 50),("嘉兴", 51),("长春", 51),("胶州", 52),("银川", 52),
    ("张家港", 52),("三门峡", 53),("锦州", 54),("南昌", 54),("柳州", 54),("三亚", 54),
    ("自贡", 56),("吉林", 56),("阳江", 57),("泸州", 57),("西宁", 57),("宜宾", 58),
    ("呼和浩特", 58),("成都", 58),("大同", 58),("镇江", 59),("桂林", 59),("张家界", 59),
    ("宜兴", 59),("北海", 60),("西安", 61),("金坛", 62),("东营", 62),("牡丹江", 63),
    ("遵义", 63),("绍兴", 63),("扬州", 64),("常州", 64),("潍坊", 65),("重庆", 66),
    ("台州", 67),("南京", 67),("滨州", 70),("贵阳", 71),("无锡", 71),("本溪", 71),
    ("克拉玛依", 72),("渭南", 72),("马鞍山", 72),("宝鸡", 72),("焦作", 75),("句容", 75),
    ("北京", 79),("徐州", 79),("衡水", 80),("包头", 80),("绵阳", 80),("乌鲁木齐", 84),
    ("枣庄", 84),("杭州", 84),("淄博", 85),("鞍山", 86),("溧阳", 86),("库尔勒", 86),
    ("安阳", 90),("开封", 90),("济南", 92),("德阳", 93),("温州", 95),("九江", 96),
    ("邯郸", 98),("临安", 99),("兰州", 99),("沧州", 100),("临沂", 103),("南充", 104),
    ("天津", 105),("富阳", 106),("泰安", 112),("诸暨", 112),("郑州", 113),("哈尔滨", 114),
    ("聊城", 116),("芜湖", 117),("唐山", 119),("平顶山", 119),("邢台", 119),("德州", 120),
    ("济宁", 120),("荆州", 127),("宜昌", 130),("义乌", 132),("丽水", 133),("洛阳", 134),
    ("秦皇岛", 136),("株洲", 143),("石家庄", 147),("莱芜", 148),("常德", 152),("保定", 153),
    ("湘潭", 154),("金华", 157),("岳阳", 169),("长沙", 175),("衢州", 177),("廊坊", 193),
    ("菏泽", 194),("合肥", 229),("武汉", 273),("大庆", 279)]

geo = Geo("全国主要城市空气质量", "data from pm2.5", title_color="#fff", title_pos="center",
width=1200, height=600, background_color='#404a59')
attr, value = geo.cast(data)
geo.add("", attr, value, visual_range=[0, 200], visual_text_color="#fff", symbol_size=15, is_visualmap=True)
geo.show_config()
geo.render_notebook()

{'_index_flag': 1874900,
 'backgroundColor': '#404a59',
 'color': ['#c23531',
           '#2f4554',
           '#61a0a8',
           '#d48265',
           '#749f83',
           '#ca8622',
           '#bda29a',
           '#6e7074',
           '#546570',
           '#c4ccd3',
           '#f05b72',
           '#ef5b9c',
           '#f47920',
           '#905a3d',
           '#fab27b',
           '#2a5caa',
           '#444693',
           '#726930',
           '#b2d235',
           '#6d8346',
           '#ac6767',
           '#1d953f',
           '#6950a1',
           '#918597',
           '#f6f5ec'],
 'geo': {'itemStyle': {'emphasis': {'areaColor': '#2a333d'},
                       'normal': {'areaColor': '#323c48',
                                  'borderColor': '#111'}},
         'label': {},
         'map': 'china'},
 'legend': [{'data': [''],
             'left': 'center',
             'orient': 'horizontal',
             'selectedMode': True,
             'show': True,
          